In [54]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.37 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/112.6 GB disk)


In [55]:
import os
import shutil
import random
from tqdm.notebook import tqdm  # Correcta importación para usar tqdm en Jupyter Notebook.
from google.colab import drive  # Correcto para interactuar con Google Drive en Google Colab.

In [56]:
!pip install tqdm --upgrade

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import os

# Listar archivos en el directorio YOLODataset
#image_dir = '/content/drive/MyDrive/fotos etiquetadas/YOLODataset'
image_dir = '/content/drive/MyDrive/Practicas_2_2024_BGH/fotos etiquetadas/YOLODataset'
print(os.listdir(image_dir))

['images', 'labels', 'test', 'dataset.yaml']


Predicción


In [59]:
!pip install ultralytics

In [60]:
import os
import random
import shutil

# Definir las rutas de tus carpetas
image_dir = "/content/drive/MyDrive/Practicas_2_2024_BGH/fotos etiquetadas/YOLODataset/images"  # Ruta de las imágenes
label_dir = "/content/drive/MyDrive/Practicas_2_2024_BGH/fotos etiquetadas/YOLODataset/labels"  # Ruta de las etiquetas

train_image_dir = os.path.join(image_dir, 'train')
val_image_dir = os.path.join(image_dir, 'val')
train_label_dir = os.path.join(label_dir, 'train')
val_label_dir = os.path.join(label_dir, 'val')

# Crear las carpetas si no existen
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# Obtener la lista de todas las imágenes
images = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Dividir en 80% entrenamiento y 20% validación
random.shuffle(images)  # Barajar las imágenes para asegurar una distribución aleatoria
split_idx = int(len(images) * 0.8)  # El 80% de las imágenes

train_images = images[:split_idx]
val_images = images[split_idx:]

# Mover las imágenes y las etiquetas a las carpetas correspondientes
for img in train_images:
    # Mover imagen
    shutil.move(os.path.join(image_dir, img), os.path.join(train_image_dir, img))

    # Mover la etiqueta correspondiente
    label = img.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')
    shutil.move(os.path.join(label_dir, label), os.path.join(train_label_dir, label))

for img in val_images:
    # Mover imagen
    shutil.move(os.path.join(image_dir, img), os.path.join(val_image_dir, img))

    # Mover la etiqueta correspondiente
    label = img.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')
    shutil.move(os.path.join(label_dir, label), os.path.join(val_label_dir, label))

print("División completada: 80% entrenamiento y 20% validación.")



División completada: 80% entrenamiento y 20% validación.


In [61]:
!yolo train data="/content/drive/MyDrive/Practicas_2_2024_BGH/fotos etiquetadas/YOLODataset/dataset.yaml" model=yolov8n.pt epochs=80

Ultralytics 8.3.37 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Practicas_2_2024_BGH/fotos etiquetadas/YOLODataset/dataset.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

In [63]:
# Buscar el archivo .pt en el entorno local
search_dir = '/content/runs/detect/'
for root, dirs, files in os.walk(search_dir):
    if 'best.pt' in files:
        print(f"Archivo encontrado en: {root}/best.pt")

Archivo encontrado en: /content/runs/detect/train5/weights/best.pt
Archivo encontrado en: /content/runs/detect/train4/weights/best.pt
Archivo encontrado en: /content/runs/detect/train7/weights/best.pt
Archivo encontrado en: /content/runs/detect/train3/weights/best.pt


In [64]:
pip install opencv-contrib-python

In [65]:
from ultralytics import YOLO
import os
import cv2
from google.colab.patches import cv2_imshow

# Cargar el modelo entrenado
model = YOLO("/content/yolov8n.pt")
#model = YOLO("/content/runs/detect/train/weights/best.pt")

# Ruta de la carpeta con las imágenes de test
test_images_path = "/content/drive/MyDrive/Practicas_2_2024_BGH/fotos etiquetadas/YOLODataset/test"

# Definir las clases esperadas
expected_classes = ['N IN 1', 'NTC 1', 'C 496', 'CN 12', 'CN 19', 'C 497', 'C 498', 'CN 15', 'CN 14', 'C 259', 'C 251', 'CN 18', 'SW 1']

# Para almacenar las detecciones "Good" y "No Good"
good_detections = []
no_good_detections = []

# Procesar las imágenes
for img_name in os.listdir(test_images_path):
    img_path = os.path.join(test_images_path, img_name)

    # Leer la imagen
    img = cv2.imread(img_path)

    # Realizar la inferencia
    results = model(img_path)

    # Obtener las predicciones
    predictions = results[0].boxes
    detected_classes = results[0].names

    # Verificar las clases detectadas
    detected_class_ids = predictions.cls.tolist()  # Los IDs de las clases detectadas
    detected_class_names = [detected_classes[int(class_id)] for class_id in detected_class_ids]

    # Calcular las clases faltantes
    detected_classes_set = set(detected_class_names)
    missing_classes = [cls for cls in expected_classes if cls not in detected_classes_set]

    # Calcular el porcentaje de faltantes
    missing_percentage = (len(missing_classes) / len(expected_classes)) * 100

    # Dibujar las cajas de detección en la imagen
    for box, class_name in zip(predictions.xyxy.tolist(), detected_class_names):
        if class_name in detected_class_names:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, class_name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar la imagen con recuadros
    print(f"Imagen: {img_name}")
    print(f"Componentes faltantes: {missing_classes}")
    print(f"Porcentaje de faltantes: {missing_percentage:.2f}%\n")

    # Mostrar la imagen con las detecciones y faltantes visualmente
    cv2_imshow(img)  # Mostrar la imagen con OpenCV en Colab

    # Almacenar las detecciones
    if missing_percentage == 0:
        good_detections.append((img_name, detected_class_names))
    else:
        no_good_detections.append((img_name, missing_classes, missing_percentage))

# Imprimir un resumen de las detecciones "Good" y "No Good"
print("Detecciones Good:")
for good in good_detections:
    print(f"Imagen: {good[0]}, Clases detectadas: {good[1]}")

print("\nDetecciones No Good:")
for no_good in no_good_detections:
    print(f"Imagen: {no_good[0]}, Componentes faltantes: {no_good[1]}, Porcentaje de faltantes: {no_good[2]:.2f}%")

Output hidden; open in https://colab.research.google.com to view.